In [20]:
import numpy as np #numpy for board positions
import pickle #to train the machine and save policy

In [21]:
rows = 3 #for set up for the playing t-t-t grid
columns = 3

In [22]:
class State: #state of the game grid, with both the player's and the machine's positions
    def __init__(self, Player1, Player2): #initialize the game grid
        self.board = np.zeros((rows, columns))
        self.Player1 = Player1
        self.Player2 = Player2
        self.isEnd = False
        self.boardHash = None
        # Player1 is the machine that plays first
        self.playerSymbol = 1 #controls whose turn it is. Updates throughout the game with '1' when P1 makes a move and '2' when P2 makes a move
    
    # get unique hash of current board state
    def getHash(self): #stores the current board 
        self.boardHash = str(self.board.reshape(columns*rows)) #updates the playing grid with the player moves
        return self.boardHash
    
    def emptySpots(self): #after each turn check which grid spots are empty
        positions = []
        for i in range(rows):
            for j in range(columns):
                if self.board[i, j] == 0:
                    positions.append((i, j)) #Appends any empty spots to the "positions" list in order to let the machine know where it can make next moves
        return positions
    
    def updateState(self, position):  #updates the game board with chosen player moves. Regulates turns.
        self.board[position] = self.playerSymbol
        # switch to another player
        self.playerSymbol = -1 if self.playerSymbol == 1 else 1 
        
    def winner(self): #Checks if the game ha ended. Assigns the winner.
        # rows
        for i in range(rows):
            if sum(self.board[i, :]) == 3: #if any row adds up to 3, then player 1 has won. Game end.
                self.isEnd = True
                return 1
            if sum(self.board[i, :]) == -3: #if any row adds up to -3, then player 2 has won. Game end.
                self.isEnd = True
                return -1
            
        # columns
        for i in range(columns):
            if sum(self.board[:, i]) == 3:#if any col adds up to 3, then player 1 has won. Game end.
                self.isEnd = True
                return 1
            if sum(self.board[:, i]) == -3:#if any col adds up to -3, then player 1 has won. Game end.
                self.isEnd = True
                return -1
            
        # diagonal
        diag_sum1 = sum([self.board[i, i] for i in range(columns)]) #checks column sum [(0,0), (1,1),(2,2)]
        diag_sum2 = sum([self.board[i, columns-i-1] for i in range(columns)]) #(0,2)(1,1)(2,0)
        diag_sum = max(diag_sum1, diag_sum2)
        
        if diag_sum == 3:  #Player 1 wins
            self.isEnd = True
            return 1
        if diag_sum == -3: #Player 2 wins
            self.isEnd = True
            return -1
        
        # tie
        # no available positions
        if len(self.emptySpots()) == 0: #no more empty spots on the grid = no possible winning combos
            self.isEnd = True
            return 0 #returns neither 1 nor -1, thus no one wins
        # not end
        self.isEnd = False #Game end.
        return None
    

    
    # only when game ends
    def giveReward(self):
        result = self.winner()
        # backpropagate reward
        if result == 1: #Machine wins
            self.Player1.feedReward(1)
            self.Player2.feedReward(0)
        elif result == -1: #Human Player wins
            self.Player1.feedReward(0)
            self.Player2.feedReward(1)
        else: #tie
            self.Player1.feedReward(0.1)
            self.Player2.feedReward(0.5)
    
    # board reset
    def reset(self):
        self.board = np.zeros((rows, columns))
        self.boardHash = None
        self.isEnd = False
        self.playerSymbol = 1
    
    #training the machine by letting it play against itself
    def play(self, rounds=100):
        for i in range(rounds):
            if i%1000 == 0:
                print("Rounds {}".format(i))
            while not self.isEnd:
                # Player 1
                positions = self.emptySpots() #look for available positions
                Player1_action = self.Player1.chooseAction(positions, self.board, self.playerSymbol)
                #choose the best action
                # take action and upate board state
                self.updateState(Player1_action) #update the board
                board_hash = self.getHash()
                self.Player1.addState(board_hash)
                # check board status if it is end

                win = self.winner()
                if win is not None:
                    # ended with Player1 either win or draw
                    self.giveReward()
                    self.Player1.reset()
                    self.Player2.reset()
                    self.reset()
                    break

                else:
                    # Player 2
                    positions = self.emptySpots()
                    Player2_action = self.Player2.chooseAction(positions, self.board, self.playerSymbol)
                    self.updateState(Player2_action)
                    board_hash = self.getHash()
                    self.Player2.addState(board_hash)
                    
                    win = self.winner()
                    if win is not None:
                        # ended with Player2 either win or draw
                        self.giveReward()
                        self.Player1.reset()
                        self.Player2.reset()
                        self.reset()
                        break
    
    # play with human
    #mostly the same as in machine play but ensures that the machine goes first
    def play2(self):
        while not self.isEnd:
            # Player 1
            positions = self.emptySpots()
            Player1_action = self.Player1.chooseAction(positions, self.board, self.playerSymbol)
            # take action and upate board state
            self.updateState(Player1_action)
            self.showBoard()
            # check board status if it is end
            win = self.winner()
            if win is not None:
                if win == 1:
                    print(self.Player1.name, "wins!")
                else:
                    print("tie!")
                self.reset()
                break

            else:
                # Player 2
                positions = self.emptySpots()
                Player2_action = self.Player2.chooseAction(positions)

                self.updateState(Player2_action)
                self.showBoard()
                win = self.winner()
                if win is not None:
                    if win == -1:
                        print(self.Player2.name, "wins!")
                    else:
                        print("tie!")
                    self.reset()
                    break

    def showBoard(self):
        # Player1: x  Player2: o
        for i in range(0, rows):
            print('-------------')
            out = '| '
            for j in range(0, columns):
                if self.board[i, j] == 1:
                    token = 'x'
                if self.board[i, j] == -1:
                    token = 'o'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('-------------')

In [23]:
class MachinePlayer: # the AI opponent
    def __init__(self, name, exp_rate=0.3): 
        #exploration vs exploitation rate --> 70% machine will take greedy action based on estimation of reward,
        #other 30% will be random
        
        self.name = name
        self.states = []  # record all positions taken in this list
        self.lr = 0.2 #learning rate (0-1). Controls how quickly the model is adapted to the problem.
        self.exp_rate = exp_rate
        self.decay_gamma = 0.9 #decay rate between 0 and 1. How much machine cares about future reward
        #1 = machine equally cares about all future rewards
        #0 = agent cares about only the reward of the current state
        self.states_value = {}  # state -> value; updates corresponding to states
    
    def getHash(self, board):
        boardHash = str(board.reshape(columns*rows))
        return boardHash
    
    #store the hash of board state into state-value dictnary
    #exploitation
    #hash the next state and choose the action with next max reward 
    def chooseAction(self, positions, current_board, symbol):
        if np.random.uniform(0, 1) <= self.exp_rate: #choose a random square on the playing grid to start the game
            # take random action
            idx = np.random.choice(len(positions))
            action = positions[idx]
        else:
            value_max = -999
            for p in positions:
                next_board = current_board.copy()
                next_board[p] = symbol
                next_boardHash = self.getHash(next_board)
                value = 0 if self.states_value.get(next_boardHash) is None else self.states_value.get(next_boardHash)
                # print("value", value)
                if value >= value_max:
                    value_max = value
                    action = p
        # print("{} takes action {}".format(self.name, action))
        return action
    
    # append a hash state
    def addState(self, state):
        self.states.append(state)
    
    # at the end of game, backpropagate and update states value
    # "the updated value of state t equals the current value of state t adding the difference between the value 
    #of next state and the value of current state, which is multiplied by a learning rate α (Given the reward of 
    #intermediate state is 0)"
    
    #positions stored in self.states
    #at the end of the game, estimates are updated in 'reversed' fashion
    def feedReward(self, reward):
        for st in reversed(self.states):
            if self.states_value.get(st) is None:
                self.states_value[st] = 0
            self.states_value[st] += self.lr*(self.decay_gamma*reward - self.states_value[st])
            reward = self.states_value[st]
            
    def reset(self):
        self.states = []
 
 #save the policy the machine learned during training in order to later apply it to match against human
    def savePolicy(self):
        fw = open('policy_' + str(self.name), 'wb')
        pickle.dump(self.states_value, fw)
        fw.close()

    def loadPolicy(self, file):
        fr = open(file,'rb')
        self.states_value = pickle.load(fr)
        fr.close()

class HumanPlayer: #human class to play against the machine
    def __init__(self, name):
        self.name = name 
    
    def chooseAction(self, positions): #ask human to input their actions
        while True:
            row = int(input("Input your action row:"))
            col = int(input("Input your action col:"))
            action = (row, col)
            if action in positions: #ask again if they choose an already occupied slot
                return action
    
    # append a hash state
    def addState(self, state):
        pass
    
    # at the end of game, backpropagate and update states value
    def feedReward(self, reward):
        pass
            
    def reset(self):
        pass

In [24]:
p1 = MachinePlayer("p1")  #train the machine against itself
p2 = MachinePlayer("p2")

st = State(p1, p2)
print("Training")
st.play(50000) #play against itself 50,000 times

Training
Rounds 0
Rounds 1000
Rounds 2000
Rounds 3000
Rounds 4000
Rounds 5000
Rounds 6000
Rounds 7000
Rounds 8000
Rounds 9000
Rounds 10000
Rounds 11000
Rounds 12000
Rounds 13000
Rounds 14000
Rounds 15000
Rounds 16000
Rounds 17000
Rounds 18000
Rounds 19000
Rounds 20000
Rounds 21000
Rounds 22000
Rounds 23000
Rounds 24000
Rounds 25000
Rounds 26000
Rounds 27000
Rounds 28000
Rounds 29000
Rounds 30000
Rounds 31000
Rounds 32000
Rounds 33000
Rounds 34000
Rounds 35000
Rounds 36000
Rounds 37000
Rounds 38000
Rounds 39000
Rounds 40000
Rounds 41000
Rounds 42000
Rounds 43000
Rounds 44000
Rounds 45000
Rounds 46000
Rounds 47000
Rounds 48000
Rounds 49000


In [25]:
p1.savePolicy() #save the training
p2.savePolicy()

In [26]:
p1.loadPolicy("policy_p1") #use training in game against human

In [27]:
p1 = MachinePlayer("computer", exp_rate=0)
p1.loadPolicy("policy_p1")

p2 = HumanPlayer("human")

st = State(p1, p2)
st.play2()


-------------
| x |   |   | 
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
Input your action row:1
Input your action col:1
-------------
| x |   |   | 
-------------
|   | o |   | 
-------------
|   |   |   | 
-------------
-------------
| x |   |   | 
-------------
| x | o |   | 
-------------
|   |   |   | 
-------------
Input your action row:2
Input your action col:0
-------------
| x |   |   | 
-------------
| x | o |   | 
-------------
| o |   |   | 
-------------
-------------
| x |   | x | 
-------------
| x | o |   | 
-------------
| o |   |   | 
-------------
Input your action row:0
Input your action col:1
-------------
| x | o | x | 
-------------
| x | o |   | 
-------------
| o |   |   | 
-------------
-------------
| x | o | x | 
-------------
| x | o |   | 
-------------
| o | x |   | 
-------------
Input your action row:1
Input your action col:1
Input your action row:1
Input your action col:2
-------------
| x | o | x | 
-------------
| x | o | 